In [1]:
data_flag = 'opp115'

# ohsumed: 23986
# opp115 3399
# reuters 10788

In [2]:
import pickle
import pandas as pd
import numpy as np
import utilities
import preprocess
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.model_selection import train_test_split

## parameters

In [3]:
# algorithm parameters
balance_ratio = 0.5
random_state = 1
threshold_factor = 1.5
test_size = 0.2

sim_type = 'cosine'
embedding_method = 'distiluse-base-multilingual-cased-v1' # try different embeddings and find proper one

np.random.seed(random_state)

data_paths = {'opp115'   : r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv',
              'ohsumed'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\ohsumed.csv',
              'reuters'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\Reuters21578.csv'}

unlabaled_ratios = {'opp115':0.75, 'ohsumed':0.95, 'reuters':0.9}

In [4]:
import warnings
warnings.filterwarnings("ignore")

# main

In [5]:
def main(data):
    print('*'*100)
    print('\x1b[1;31m'+data+'\x1b[0m')
    # reading data
    df = utilities.read_data(data_paths[data])
    X = df['text'].apply(preprocess.preprocess_text)
    y = df.drop(['text'], axis=1)
    # ------------------------------------------------------------------------------------------------------------------------------
    # reading from a pickle instead of applying vectorization
    X_num = utilities.vectorize_data(X, embedding_method)
    X_num = pd.Series([np.squeeze(i) for i in X_num])
    X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabaled_ratios[data], 
                                                                  random_state=random_state)
    
    print(X_labeled.shape, y_labeled.shape, X_unlabeled.shape, y_unlabeled.shape)
    utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test)
    #utilities.classifier(np.vstack(X_labeled.values), y_labeled, np.vstack(X_test.values), y_test)
    # -----------------------------------------------------------------------------------------------------------------------------
    # calculation number of instances to balance dataset
    balance_ratio = 0.5
    num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio)
    # -----------------------------------------------------------------------------------------------------------------------------
    # oversampling dataset using unlabeled data with the given ratios
    validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset(num_of_new_instances, 
                                                                                              X_labeled, y_labeled,
                                                                                              X_unlabeled, y_unlabeled, 
                                                                                              X_test, y_test, 
                                                                                              sim_calculation_type='safe_interval', 
                                                                                              batch_size=5)
        
    # -----------------------------------------------------------------------------------------------------------------------------
    # check if the result gets better
    print(X_labeled.shape, X_unlabeled.shape, X_test.shape)
    utilities.multilabel_classifier(np.vstack(X_labeled.values), y_labeled, np.vstack(X_test.values), y_test)  
    
    # comparing the found labels and ground truth
    y_true, y_pred = [], []
    for _, _, _, y_t, y_p in validation:
        y_true.append(list(y_t.values))
        y_pred.append(list(y_p.values()))
    
    acc = 1-hamming_loss(y_true, y_pred)
    emr = accuracy_score(y_true, y_pred)  
    print('-'*30)
    print(f'Exact match ratio : {emr:.2f} ')
    print(f'Accuracy          : {acc:.2f} ')
    print('-'*30)
    
    print('*'*100)
    print('/'*100)
    print('*'*100)

In [6]:
for data in data_paths.keys():
    main(data)

****************************************************************************************************
opp115
(679,) (679, 12) (2040,) (2040, 12)
| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | 
Multilabel Classifier Results
LinearSVM
------------------------------
hamLoss: 0.07
Exact Match Ratio: 0.46
------------------------------
Classification Report
                                      precision    recall  f1-score   support

                      Data Retention       0.11      0.23      0.15        13
                       Data Security       0.69      0.88      0.77        40
                        Do Not Track       0.86      1.00      0.92         6
          First Party Collection/Use       0.73      0.88      0.80       230
International and Specific Audiences       0.92      0.81      0.86        68
                Introductory/Generic       0.50      0.70      0.59        76
                       Policy Change       0.7

f1 before : 0.923076923076923  f1 after : 0.6666666666666667
Shapes --------------
(679,) (1710,)
f1 before : 0.923076923076923  f1 after : 0.7142857142857143
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.923076923076923  f1 after : 0.923076923076923
Shapes --------------
(679,) (1707,)
f1 before : 0.92307692307692

f1 before : 0.8510638297872342  f1 after : 0.6779661016949153
Shapes --------------
(684,) (1627,)
f1 before : 0.8510638297872342  f1 after : 0.7241379310344828
Shapes --------------
(684,) (1622,)
f1 before : 0.8510638297872342  f1 after : 0.7636363636363637
Shapes --------------
(684,) (1617,)
f1 before : 0.8510638297872342  f1 after : 0.7142857142857143
Shapes --------------
(684,) (1612,)
f1 before : 0.8510638297872342  f1 after : 0.6349206349206349
Shapes --------------
(684,) (1607,)
f1 before : 0.8510638297872342  f1 after : 0.7692307692307692
Shapes --------------
(684,) (1602,)
f1 before : 0.8510638297872342  f1 after : 0.6909090909090909
Shapes --------------
(684,) (1597,)
f1 before : 0.8510638297872342  f1 after : 0.7142857142857143
Shapes --------------
(684,) (1592,)
f1 before : 0.8510638297872342  f1 after : 0.7407407407407406
Shapes --------------
(684,) (1587,)
f1 before : 0.8510638297872342  f1 after : 0.7037037037037037
Shapes --------------
(684,) (1582,)
f1 before 

f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before : 0.8510638297872342  f1 after : 0.8510638297872342
Shapes --------------
(684,) (1387,)
f1 before 

f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before : 0.1904761904761905  f1 after : 0.1904761904761905
Shapes --------------
(719,) (1213,)
f1 before 

...

f1 before : 0.46274509803921565  f1 after : 0.46386333771353483
adding instances ... 
Shapes --------------
(1961,) (9167,)
f1 before : 0.46386333771353483  f1 after : 0.46367239101717306
Shapes --------------
(1961,) (9162,)
f1 before : 0.46386333771353483  f1 after : 0.465300727032386
adding instances ... 
Shapes --------------
(1966,) (9157,)
f1 before : 0.465300727032386  f1 after : 0.4637873754152824
Shapes --------------
(1966,) (9152,)
f1 before : 0.465300727032386  f1 after : 0.4663143989431969
adding instances ... 
Shapes --------------
(1971,) (9147,)
f1 before : 0.4663143989431969  f1 after : 0.46041250831669994
Shapes --------------
(1971,) (9142,)
f1 before : 0.4663143989431969  f1 after : 0.46825396825396826
adding instances ... 
Shapes --------------
(1976,) (9137,)
f1 before : 0.46825396825396826  f1 after : 0.46794448116325177
Shapes --------------
(1976,) (9132,)
f1 before : 0.46825396825396826  f1 after : 0.46697798532354906
Shapes --------------
(1976,) (9127,)
f1 b

f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes --------------
(2016,) (9032,)
f1 before : 0.48816768086544965  f1 after : 0.48816768086544965
Shapes -----------

f1 before : 0.5850178359096314  f1 after : 0.5828367103694874
Shapes --------------
(2086,) (8897,)
f1 before : 0.5850178359096314  f1 after : 0.5890083632019116
adding instances ... 
Shapes --------------
(2091,) (8892,)
f1 before : 0.5890083632019116  f1 after : 0.5884455032757595
Shapes --------------
(2091,) (8887,)
f1 before : 0.5890083632019116  f1 after : 0.5890083632019116
Shapes --------------
(2091,) (8882,)
f1 before : 0.5890083632019116  f1 after : 0.5856550088915234
Shapes --------------
(2091,) (8877,)
f1 before : 0.5890083632019116  f1 after : 0.5899280575539568
adding instances ... 
Shapes --------------
(2096,) (8872,)
f1 before : 0.5899280575539568  f1 after : 0.5883054892601431
Shapes --------------
(2096,) (8867,)
f1 before : 0.5899280575539568  f1 after : 0.5914817036592681
adding instances ... 
Shapes --------------
(2101,) (8862,)
f1 before : 0.5914817036592681  f1 after : 0.5913461538461539
Shapes --------------
(2101,) (8857,)
f1 before : 0.5914817036592681  f1

f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after : 0.60984393757503
Shapes --------------
(2166,) (8556,)
f1 before : 0.60984393757503  f1 after :

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
stop

In [ ]:
from nltk.corpus import words
all_words = words.words()

In [ ]:
words_5len = [word.lower() for word in all_words if len(word)==5]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data_vecs = vectorizer.fit_transform(data_words).toarray()

In [ ]:
for word in words_5len:
    if 'h' in word and 'k' in word and 'n' in word:
        print(word)

In [ ]:
from gensim.corpora.dictionary import Dictionary
common_dictionary = Dictionary(data_words)
common_corpus = [common_dictionary.doc2bow(text) for text in data_words]

In [ ]:
common_corpus

In [ ]:
data_vecs[0]

In [ ]:
data_vecs

In [ ]:
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div

In [ ]:
for word in words_5len:
    if word.startswith('se') and 'r' in word:
        print(word)

In [ ]:
letter_0 = ''
letter_1 = ''
letter_2 = 'a'
letter_3 = ''
letter_4 = ''

exist_letters = 'acs'
banned_letters = 'trdefou'

In [ ]:
def KL_divergence(p, q):
        """ Compute KL divergence of two vectors, K(p || q)."""
        return sum(p[x] * log((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)

In [ ]:
from numpy import zeros, array
from math import sqrt, log

In [ ]:
jensenshannon(p, q)

In [ ]:
filtered = [word for word in words_5len for e in exist_letters if e in word]
filtered

In [ ]:
filtered = [word for word in filtered for b in banned_letters if b in word]
filtered = [word for word in filtered if letter_0 and word[0]==letter_0]
filtered = [word for word in filtered if letter_1 and word[0]==letter_1]
filtered = [word for word in filtered if letter_2 and word[0]==letter_2]
filtered = [word for word in filtered if letter_3 and word[0]==letter_3]
filtered = [word for word in filtered if letter_4 and word[0]==letter_4]
filtered

In [ ]:
class JSD(object):
    def __init__(self):
        self.log2 = log(2)


    def KL_divergence(self, p, q):
        """ Compute KL divergence of two vectors, K(p || q)."""
        return sum(p[x] * log((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)

    def Jensen_Shannon_divergence(self, p, q):
        """ Returns the Jensen-Shannon divergence. """
        self.JSD = 0.0
        weight = 0.5
        average = zeros(len(p)) #Average
        for x in range(len(p)):
            average[x] = weight * p[x] + (1 - weight) * q[x]
            self.JSD = (weight * self.KL_divergence(array(p), average)) + ((1 - weight) * self.KL_divergence(array(q), average))
        return 1-(self.JSD/sqrt(2 * self.log2))

if __name__ == '__main__':
    J = JSD()
    p = [1.0/10, 9.0/10, 0]
    q = [0, 1.0/10, 9.0/10]
    p = data_vecs[0]
    q = data_vecs[1]
    print(J.Jensen_Shannon_divergence(p, q))

In [ ]:
if letter_2:
    print('a')

In [ ]:
for word in words_5len:
    if 'o' in word and 'u' in word and 'a' not in word and 'i' not in word and 'd' not in word:
        print(word)